##Prediccion Inmuebles

In [22]:
# Como siempre, primero nos importamos las librerías con las que vamos a estar trabajando

import requests # Nos va a permitir hacer llamados con el protocolo HTTP
import json # Nos va a permitir trabajar con los archivos JSON
import pandas as pd # Nuestra librería amiga de siempre que nos permite trabajar con datos tabulares
import numpy as np # numpy para trabajar con operaciones matemáticas y arrays

In [23]:
# Nos definimos una función que nos va a dejar acceder a la información que nos devuelva el URL que pongamos
# en el argumento. Recuerden que el URL es la dirección del sitio web que vamos a usar

def get_response(url):
    try:
        response = requests.get(url, timeout=3)
        response.raise_for_status()
        return response
    except requests.exceptions.HTTPError as errh:
        print ("Http Error:",errh)
    except requests.exceptions.ConnectionError as errc:
        print ("Error Connecting:",errc)
    except requests.exceptions.Timeout as errt:
        print ("Timeout Error:",errt)
    except requests.exceptions.RequestException as err:
        print ("OOps: Something Else",err)

In [24]:

# Primero indicamos la consulta que vamos a hacer
#Rosario y alrededores
url='https://api.mercadolibre.com/sites/MLA/search?item_location=lat:-33.01_-32.90,lon:-60.771_-60.618&category=MLA401685&limit=50&offset=0#json'

offset=0 #Arrancamos con el offset 0
r = get_response(url) #Vemos los resultados de la consulta
maximum = int(str(r.json()["paging"]["total"])) #Consultamos cuál es el número máximo de resultados

print('Encontramos {} resultados para nuestra consulta'.format(maximum))

Encontramos 2544 resultados para nuestra consulta


In [25]:

# Vamos a forzarnos a buscar pocos resultados para seguir testeando y no perder tanta velocidad
# Cuando vayamos full mode, esta linea de abajo hay que comentarla o borrarla
maximum=1000


# Ahora, sabiendo esto, vamos a traer todos los resultados iterando y aumentando el offset en cada loop.
# Recuerden que solo podemos traer de a tandas de a 50 resultados
item_list = []
while r.status_code == 200 and offset <= maximum:
    r = get_response(url)
    data = r.json()
    length = len(data['results'])
    for i in range(length):
        item_id = data['results'][i]['id']
        item_list.append(item_id)
    offset += 50
    print("Porcentaje de completitud: {:0.2%}".format(offset/maximum),end='\r')
        
len(item_list)

1050

In [26]:
#Veamos que hay en el listado de items
item_list[0:10]

['MLA925206631',
 'MLA925169953',
 'MLA925174110',
 'MLA925133739',
 'MLA925179712',
 'MLA925840993',
 'MLA914946745',
 'MLA915221069',
 'MLA871241882',
 'MLA917040007']

In [27]:
#Veamos un item en particular
print("https://api.mercadolibre.com/items/{}".format(item_list[0]))
item_add=requests.get("https://api.mercadolibre.com/items/"+item_list[0])
item_add.json()

https://api.mercadolibre.com/items/MLA925206631


{'id': 'MLA925206631',
 'site_id': 'MLA',
 'title': 'Casa En Fisherton',
 'subtitle': None,
 'seller_id': 452912735,
 'category_id': 'MLA401685',
 'official_store_id': None,
 'price': 90000,
 'base_price': 90000,
 'original_price': None,
 'currency_id': 'USD',
 'initial_quantity': 1,
 'available_quantity': 1,
 'sold_quantity': 0,
 'sale_terms': [{'id': 'IS_FINANCEABLE',
   'name': 'Es financiable',
   'value_id': '242084',
   'value_name': 'No',
   'value_struct': None,
   'values': [{'id': '242084', 'name': 'No', 'struct': None}]}],
 'buying_mode': 'classified',
 'listing_type_id': 'gold_premium',
 'start_time': '2021-06-12T02:38:35.000Z',
 'stop_time': '2021-08-10T04:00:00.000Z',
 'condition': 'used',
 'permalink': 'https://casa.mercadolibre.com.ar/MLA-925206631-casa-en-fisherton-_JM',
 'thumbnail_id': '979887-MLA46370432337_062021',
 'thumbnail': 'http://http2.mlstatic.com/D_979887-MLA46370432337_062021-I.jpg',
 'secure_thumbnail': 'https://http2.mlstatic.com/D_979887-MLA46370432337

In [28]:
#Podemos guardar el json en el disco
out_file = open("myfile.json", "w") 
json.dump(item_add.json(), out_file, indent = 6) 
out_file.close()

In [29]:
#Ahora, teniendo el listado de los items, nos vamos a traer las características de ellos
final_list = []
for i in range(len(item_list)):
    item="https://api.mercadolibre.com/items/{}".format(item_list[i])
    item_add = requests.get(item)
    item_add = item_add.json()
    final_list.append(item_add)
    print("Porcentaje de completitud: {:0.2%}".format((i+1)/len(item_list)),end='\r')

In [30]:
#Veamos el primer elemento
final_list[0]

{'id': 'MLA925206631',
 'site_id': 'MLA',
 'title': 'Casa En Fisherton',
 'subtitle': None,
 'seller_id': 452912735,
 'category_id': 'MLA401685',
 'official_store_id': None,
 'price': 90000,
 'base_price': 90000,
 'original_price': None,
 'currency_id': 'USD',
 'initial_quantity': 1,
 'available_quantity': 1,
 'sold_quantity': 0,
 'sale_terms': [{'id': 'IS_FINANCEABLE',
   'name': 'Es financiable',
   'value_id': '242084',
   'value_name': 'No',
   'value_struct': None,
   'values': [{'id': '242084', 'name': 'No', 'struct': None}]}],
 'buying_mode': 'classified',
 'listing_type_id': 'gold_premium',
 'start_time': '2021-06-12T02:38:35.000Z',
 'stop_time': '2021-08-10T04:00:00.000Z',
 'condition': 'used',
 'permalink': 'https://casa.mercadolibre.com.ar/MLA-925206631-casa-en-fisherton-_JM',
 'thumbnail_id': '979887-MLA46370432337_062021',
 'thumbnail': 'http://http2.mlstatic.com/D_979887-MLA46370432337_062021-I.jpg',
 'secure_thumbnail': 'https://http2.mlstatic.com/D_979887-MLA46370432337

In [31]:
#Exploremos las llaves que tiene el archivo
final_list[0].keys()

dict_keys(['id', 'site_id', 'title', 'subtitle', 'seller_id', 'category_id', 'official_store_id', 'price', 'base_price', 'original_price', 'currency_id', 'initial_quantity', 'available_quantity', 'sold_quantity', 'sale_terms', 'buying_mode', 'listing_type_id', 'start_time', 'stop_time', 'condition', 'permalink', 'thumbnail_id', 'thumbnail', 'secure_thumbnail', 'pictures', 'video_id', 'descriptions', 'accepts_mercadopago', 'non_mercado_pago_payment_methods', 'shipping', 'international_delivery_mode', 'seller_address', 'seller_contact', 'location', 'coverage_areas', 'attributes', 'warnings', 'listing_source', 'variations', 'status', 'sub_status', 'tags', 'warranty', 'catalog_product_id', 'domain_id', 'parent_item_id', 'differential_pricing', 'deal_ids', 'automatic_relist', 'date_created', 'last_updated', 'health', 'catalog_listing', 'channels'])

In [32]:
#Transformamos los datos a tabla con pd.json_normalize
data=pd.json_normalize(final_list)
data.head()

,id,site_id,title,subtitle,seller_id,category_id,official_store_id,price,base_price,original_price,...,location.neighborhood.id,location.neighborhood.name,location.city.id,location.city.name,location.state.id,location.state.name,location.country.id,location.country.name,seller_address.search_location.neighborhood.id,seller_address.search_location.neighborhood.name
0,MLA925206631,MLA,Casa En Fisherton,None,452912735,MLA401685,NaN,90000,90000,None,...,,,TUxBQ1JPUzg1Yjg3,Rosario,TUxBUFNBTmU5Nzk2,Santa Fe,AR,Argentina,NaN,NaN
1,MLA925169953,MLA,Cordoba 2354 00 03,None,452912735,MLA401685,NaN,190000,190000,None,...,TUxBQlJPUzI5MDha,Centro,TUxBQ1JPUzg1Yjg3,Rosario,TUxBUFNBTmU5Nzk2,Santa Fe,AR,Argentina,NaN,NaN
2,MLA925174110,MLA,Mitre 3063,None,452912735,MLA401685,NaN,130000,130000,None,...,,,TUxBQ1JPUzg1Yjg3,Rosario,TUxBUFNBTmU5Nzk2,Santa Fe,AR,Argentina,NaN,NaN
3,MLA925133739,MLA,San Juan 2185,None,452912735,MLA401685,NaN,360000,360000,None,...,,,TUxBQ1JPUzg1Yjg3,Rosario,TUxBUFNBTmU5Nzk2,Santa Fe,AR,Argentina,NaN,NaN
4,MLA925179712,MLA,Saavedra 1310,None,452912735,MLA401685,NaN,110000,110000,None,...,,,TUxBQ1JPUzg1Yjg3,Rosario,TUxBUFNBTmU5Nzk2,Santa Fe,AR,Argentina,NaN,NaN


In [33]:
#Veamos que hay en attributes. Si ponemos type, vamos aver que es un listado de diccionarios
#Vamos a tener que buscar el elemento que tenga m2, area o algo parecido
indices=[]

for i,s in enumerate(final_list[0]['attributes']):
    for j in s:
        if 'AREA' in str(s[j]):
            indices.append([i,s])

In [36]:
type(indices)


list

In [37]:
indices

[[27,
  {'id': 'COVERED_AREA',
   'name': 'Superficie cubierta',
   'value_id': None,
   'value_name': '80 m²',
   'value_struct': {'number': 80, 'unit': 'm²'},
   'values': [{'id': None,
     'name': '80 m²',
     'struct': {'number': 80, 'unit': 'm²'}}],
   'attribute_group_id': 'FIND',
   'attribute_group_name': 'Ficha técnica'}],
 [35,
  {'id': 'TOTAL_AREA',
   'name': 'Superficie total',
   'value_id': None,
   'value_name': '144 m²',
   'value_struct': {'number': 144, 'unit': 'm²'},
   'values': [{'id': None,
     'name': '144 m²',
     'struct': {'number': 144, 'unit': 'm²'}}],
   'attribute_group_id': 'FIND',
   'attribute_group_name': 'Ficha técnica'}]]

In [43]:
# Ahora que sabemos en que posición estan los m2, miremos para confirmarlo
final_list[0]['attributes'][35]

{'id': 'TOTAL_AREA',
 'name': 'Superficie total',
 'value_id': None,
 'value_name': '144 m²',
 'value_struct': {'number': 144, 'unit': 'm²'},
 'values': [{'id': None,
   'name': '144 m²',
   'struct': {'number': 144, 'unit': 'm²'}}],
 'attribute_group_id': 'FIND',
 'attribute_group_name': 'Ficha técnica'}

In [44]:
# Llamemos el valor de los m2 (y de paso convertimos el str a float)
pd.to_numeric(final_list[0]['attributes'][35]['value_name'][0:-2],errors='coerce')

144

In [46]:
[pd.to_numeric(elemento['attributes'][14]['value_name'][0:-2],errors='coerce') for elemento in final_list]

IndexError: list index out of range

In [48]:
#Que hay en elemento?
elemento

{'id': 'MLA923708521',
 'site_id': 'MLA',
 'title': 'Corrientes 2440 - Casa Interna - No Ph - U$s 91.000 - Venta',
 'subtitle': None,
 'seller_id': 271037183,
 'category_id': 'MLA401685',
 'official_store_id': None,
 'price': 91000,
 'base_price': 91000,
 'original_price': None,
 'currency_id': 'USD',
 'initial_quantity': 1,
 'available_quantity': 1,
 'sold_quantity': 0,
 'sale_terms': [{'id': 'IS_FINANCEABLE',
   'name': 'Es financiable',
   'value_id': '242084',
   'value_name': 'No',
   'value_struct': None,
   'values': [{'id': '242084', 'name': 'No', 'struct': None}]}],
 'buying_mode': 'classified',
 'listing_type_id': 'gold_premium',
 'start_time': '2021-06-03T13:30:41.000Z',
 'stop_time': '2021-08-08T04:00:00.000Z',
 'condition': 'not_specified',
 'permalink': 'https://casa.mercadolibre.com.ar/MLA-923708521-corrientes-2440-casa-interna-no-ph-us-91000-venta-_JM',
 'thumbnail_id': '794054-MLA46257770677_062021',
 'thumbnail': 'http://http2.mlstatic.com/D_794054-MLA46257770677_0620

In [49]:
elemento['attributes']

[{'id': 'BEDROOMS',
  'name': 'Dormitorios',
  'value_id': None,
  'value_name': '3',
  'value_struct': None,
  'values': [{'id': None, 'name': '3', 'struct': None}],
  'attribute_group_id': 'FIND',
  'attribute_group_name': 'Ficha técnica'},
 {'id': 'COVERED_AREA',
  'name': 'Superficie cubierta',
  'value_id': None,
  'value_name': '53 m²',
  'value_struct': {'number': 53, 'unit': 'm²'},
  'values': [{'id': None,
    'name': '53 m²',
    'struct': {'number': 53, 'unit': 'm²'}}],
  'attribute_group_id': 'FIND',
  'attribute_group_name': 'Ficha técnica'},
 {'id': 'FULL_BATHROOMS',
  'name': 'Baños',
  'value_id': None,
  'value_name': '1',
  'value_struct': None,
  'values': [{'id': None, 'name': '1', 'struct': None}],
  'attribute_group_id': 'FIND',
  'attribute_group_name': 'Ficha técnica'},
 {'id': 'PARKING_LOTS',
  'name': 'Cocheras',
  'value_id': None,
  'value_name': '0',
  'value_struct': None,
  'values': [{'id': None, 'name': '0', 'struct': None}],
  'attribute_group_id': 'FI

In [50]:
#¿Por que nos salta error?
listado=[]
i=0
for elemento in final_list:
    print(i)
    i+=1
    listado.append(pd.to_numeric(elemento['attributes'][14]['value_name'][0:-2],errors='coerce'))

0
1
2
3
4
5
6
7
8
9
10
11
12


IndexError: list index out of range

In [51]:
#COVERED AREA PARECE ESTA EN OTRA POSICION
indices=[]
for i,s in enumerate(elemento['attributes']):
    for j in s:
        if 'COVERED_AREA' in str(s[j]):
            indices.append([i,s])
indices

[[1,
  {'id': 'COVERED_AREA',
   'name': 'Superficie cubierta',
   'value_id': None,
   'value_name': '53 m²',
   'value_struct': {'number': 53, 'unit': 'm²'},
   'values': [{'id': None,
     'name': '53 m²',
     'struct': {'number': 53, 'unit': 'm²'}}],
   'attribute_group_id': 'FIND',
   'attribute_group_name': 'Ficha técnica'}]]